In [13]:
import mindspore as ms
from mindcv.models import create_model
from mindcv.loss import create_loss
from mindcv.optim import create_optimizer
from mindcv.data import create_transforms  # (kept if you still use it)
from mindspore import context, ops
from mindspore.dataset import ImageFolderDataset

# NEW: extra vision ops for stronger augmentation + rescaling
from mindspore.dataset.vision import (
    Decode, Resize, Normalize, HWC2CHW, Rescale,
    RandomHorizontalFlip, RandomErasing, Inter
)
from mindspore.dataset.transforms import TypeCast
import mindspore.common.dtype as mstype

import numpy as np, os, pathlib, json, time
ms.set_seed(42); np.random.seed(42)


In [14]:
# If you do have a CUDA/Ascend build, set that here; otherwise CPU is fine.
device = "GPU" if "GPU" in (context.get_context("device_target") or "CPU") else "CPU"
context.set_context(mode=ms.GRAPH_MODE, device_target=device)
print("MindSpore device target:", context.get_context("device_target"))


[WARNING] ME(6616:4660,MainProcess):2025-10-24-18:37:09.407.383 [mindspore\context.py:1412] For 'context.set_context', the parameter 'device_target' will be deprecated and removed in a future version. Please use the api mindspore.set_device() instead.


MindSpore device target: CPU


In [15]:
train_dir = "../train"
val_dir   = "../val"
assert os.path.isdir(train_dir), f"❌ Train directory not found: {train_dir}"
assert os.path.isdir(val_dir),   f"❌ Validation directory not found: {val_dir}"

# Class order used everywhere (and saved with the checkpoint)
classes = sorted([d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, d))])
num_classes = len(classes)
print("Classes:", classes, "| num_classes:", num_classes)

# Optional: show counts per class
def count_images(root, cls):
    p = os.path.join(root, cls)
    return sum(1 for f in os.listdir(p) if f.lower().endswith((".jpg",".jpeg",".png")))
print("Train counts:", {c: count_images(train_dir, c) for c in classes})
print("Val counts:  ", {c: count_images(val_dir, c) for c in classes})

dataset_train = ImageFolderDataset(dataset_dir=train_dir, shuffle=True, decode=False)
dataset_val   = ImageFolderDataset(dataset_dir=val_dir,   shuffle=False, decode=False)

print(f"✅ Loaded dataset: {dataset_train.get_dataset_size()} train, {dataset_val.get_dataset_size()} val samples.")


Classes: ['french_fries', 'hamburger', 'pancakes', 'pizza', 'sushi', 'tiramisu'] | num_classes: 6
Train counts: {'french_fries': 200, 'hamburger': 200, 'pancakes': 200, 'pizza': 200, 'sushi': 200, 'tiramisu': 200}
Val counts:   {'french_fries': 200, 'hamburger': 200, 'pancakes': 200, 'pizza': 200, 'sushi': 200, 'tiramisu': 200}
✅ Loaded dataset: 1200 train, 1200 val samples.


In [16]:
mean = [0.485, 0.456, 0.406]
std  = [0.229, 0.224, 0.225]
batch_size = 16            # try 16 or 32 if memory allows
num_workers = 4            # tune for your CPU cores

# TRAIN transforms: decode → resize → flip → color jitter → rescale(0..1) → normalize → HWC2CHW → float32 → random erasing
transforms_train = [
    Decode(),
    Resize((224, 224), interpolation=Inter.BICUBIC),
    RandomHorizontalFlip(prob=0.5),
    Rescale(1/255.0, 0.0),
    Normalize(mean=mean, std=std),
    HWC2CHW(),
    TypeCast(mstype.float32),
    RandomErasing(prob=0.25),
]


# VAL transforms: deterministic
transforms_val = [
    Decode(),
    Resize((224, 224), interpolation=Inter.BICUBIC),
    Rescale(1/255.0, 0.0),
    Normalize(mean=mean, std=std),
    HWC2CHW(),
    TypeCast(mstype.float32),
]

dataset_train = dataset_train.map(operations=transforms_train, input_columns="image",
                                  num_parallel_workers=num_workers)
dataset_train = dataset_train.batch(batch_size, drop_remainder=True)

dataset_val = dataset_val.map(operations=transforms_val, input_columns="image",
                              num_parallel_workers=num_workers)
dataset_val = dataset_val.batch(batch_size, drop_remainder=True)

print("✅ Datasets decoded, transformed, and batched.")


✅ Datasets decoded, transformed, and batched.


In [17]:
model = create_model(model_name="resnet50", num_classes=num_classes, pretrained=True)
model.set_train(True)
print("✅ Model: ResNet50 with", num_classes, "classes")

# Optional (staged training): split params for head vs. backbone
head_params = [p for p in model.trainable_params() if "classifier" in p.name]
backbone_params = [p for p in model.trainable_params() if "classifier" not in p.name]


[WARNING] ME(6616:4660,MainProcess):2025-10-24-18:37:59.610.598 [mindspore\train\serialization.py:1789] For 'load_param_into_net', 2 parameters in the 'net' are not loaded, because they are not in the 'parameter_dict', please check whether the network structure is consistent when training and loading checkpoint.
[WARNING] ME(6616:4660,MainProcess):2025-10-24-18:37:59.611.603 [mindspore\train\serialization.py:1793] ['classifier.weight', 'classifier.bias'] are not loaded.


✅ Model: ResNet50 with 6 classes


In [21]:
# ----- Manual label smoothing setup -----
num_classes = num_classes  # already defined earlier from your folders
epsilon = 0.1              # smoothing factor

# Loss takes *probability targets* (not sparse indices)
loss_fn = ms.nn.SoftmaxCrossEntropyWithLogits(sparse=False, reduction='mean')

# Convert class indices -> smoothed one-hot
def smooth_labels(labels, num_classes, eps=0.1):
    # one_hot returns float32 if on/off values are float32
    on  = ms.Tensor(1.0, ms.float32)
    off = ms.Tensor(0.0, ms.float32)
    oh = ops.one_hot(labels, num_classes, on, off).astype(ms.float32)
    return (1.0 - eps) * oh + eps / num_classes
lr_head = 1e-3
lr_full = 1e-4

head_params = [p for p in model.trainable_params() if "classifier" in p.name]
optimizer = create_optimizer(head_params if head_params else model.trainable_params(),
                             opt="adamw", lr=lr_head, weight_decay=1e-4)


In [22]:
def train_one_epoch(model, dataset, loss_fn, optimizer):
    model.set_train(True)
    total_loss = ms.Tensor(0.0, ms.float32)
    total_correct = ms.Tensor(0, ms.int32)
    total_samples = ms.Tensor(0, ms.int32)

    def forward_fn(x, y):
        logits = model(x)
        # y is class indices; convert to smoothed one-hot
        y_smooth = smooth_labels(y, num_classes, epsilon).astype(ms.float32)
        loss = loss_fn(logits, y_smooth)
        return loss, logits
    ...


    grad_fn = ops.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)

    for batch in dataset.create_dict_iterator():
        images, labels = batch["image"], batch["label"]
        (loss, logits), grads = grad_fn(images, labels)
        optimizer(grads)

        preds = ops.Argmax(axis=1)(logits)
        total_correct += ops.ReduceSum()(ops.Equal()(preds, labels).astype(ms.int32))
        total_samples += labels.shape[0]
        total_loss += loss

    avg_loss = float((total_loss / total_samples.astype(ms.float32)).asnumpy())
    acc = float((total_correct.astype(ms.float32) / total_samples.astype(ms.float32)).asnumpy())
    return avg_loss, acc

def validate(model, dataset):
    model.set_train(False)
    total_correct = 0; total_samples = 0
    for batch in dataset.create_dict_iterator():
        logits = model(batch["image"])
        preds = logits.asnumpy().argmax(axis=1)
        labels = batch["label"].asnumpy()
        total_correct += (preds == labels).sum()
        total_samples += labels.shape[0]
    return total_correct / total_samples


In [23]:
num_epochs_head = 3     # head-only warmup
num_epochs_full = 12    # full fine-tune (total 15)
best_acc, best_epoch = 0.0, -1
save_dir = pathlib.Path("../models"); save_dir.mkdir(parents=True, exist_ok=True)

print(f"🚀 Training head for {num_epochs_head} epochs...")
for epoch in range(num_epochs_head):
    t0 = time.time()
    train_loss, train_acc = train_one_epoch(model, dataset_train, loss_fn, optimizer)
    val_acc = validate(model, dataset_val)
    print(f"[Head {epoch+1}/{num_epochs_head}] "
          f"Loss:{train_loss:.4f} | Train:{train_acc:.4f} | Val:{val_acc:.4f} | {time.time()-t0:.1f}s")
    if val_acc > best_acc:
        best_acc, best_epoch = val_acc, epoch+1
        ms.save_checkpoint(model, str(save_dir / "resnet50_food_best.ckpt"))

# Switch optimizer to fine-tune ALL layers at a lower LR
optimizer = create_optimizer(model.trainable_params(), opt="adamw", lr=lr_full, weight_decay=1e-4)

print(f"\n🔧 Fine-tuning all layers for {num_epochs_full} epochs...")
for e in range(num_epochs_full):
    epoch = num_epochs_head + e + 1
    t0 = time.time()
    train_loss, train_acc = train_one_epoch(model, dataset_train, loss_fn, optimizer)
    val_acc = validate(model, dataset_val)
    print(f"[Full {e+1}/{num_epochs_full}] "
          f"Loss:{train_loss:.4f} | Train:{train_acc:.4f} | Val:{val_acc:.4f} | {time.time()-t0:.1f}s")
    if val_acc > best_acc:
        best_acc, best_epoch = val_acc, epoch
        ms.save_checkpoint(model, str(save_dir / "resnet50_food_best.ckpt"))

print(f"\n✅ Best Val Acc: {best_acc:.4f} at epoch {best_epoch}")


🚀 Training head for 3 epochs...
[Head 1/3] Loss:0.0827 | Train:0.5792 | Val:0.7958 | 89.1s


[WARNING] ME(6616:4660,MainProcess):2025-10-24-18:46:14.797.165 [mindspore\common\api.py:125] The function "after_grad" at the file "c:\Users\AJ\anaconda3\envs\project\lib\site-packages\mindspore\ops\composite\base.py", line 597 has been compiled again. Try to reuse the function object decorated by @jit to reduce the compile time. For more details, get instructions about `jit` at https://www.mindspore.cn/search?inputValue=jit. 


[Head 2/3] Loss:0.0607 | Train:0.7600 | Val:0.8508 | 83.8s


[WARNING] ME(6616:4660,MainProcess):2025-10-24-18:47:38.687.046 [mindspore\common\api.py:125] The function "after_grad" at the file "c:\Users\AJ\anaconda3\envs\project\lib\site-packages\mindspore\ops\composite\base.py", line 597 has been compiled again. Try to reuse the function object decorated by @jit to reduce the compile time. For more details, get instructions about `jit` at https://www.mindspore.cn/search?inputValue=jit. 


[Head 3/3] Loss:0.0555 | Train:0.7958 | Val:0.8692 | 84.6s


...

[WARNING] ME(6616:4660,MainProcess):2025-10-24-19:18:09.944.864 [mindspore\common\api.py:125] The function "after_grad" at the file "c:\Users\AJ\anaconda3\envs\project\lib\site-packages\mindspore\ops\composite\base.py", line 597 has been compiled again. Try to reuse the function object decorated by @jit to reduce the compile time. For more details, get instructions about `jit` at https://www.mindspore.cn/search?inputValue=jit. 


[Full 7/12] Loss:0.0280 | Train:0.9992 | Val:0.9975 | 257.3s
[Full 8/12] Loss:0.0279 | Train:0.9992 | Val:1.0000 | 253.1s


[WARNING] ME(6616:4660,MainProcess):2025-10-24-19:22:23.108.053 [mindspore\common\api.py:125] The function "after_grad" at the file "c:\Users\AJ\anaconda3\envs\project\lib\site-packages\mindspore\ops\composite\base.py", line 597 has been compiled again. Try to reuse the function object decorated by @jit to reduce the compile time. For more details, get instructions about `jit` at https://www.mindspore.cn/search?inputValue=jit. 
[WARNING] ME(6616:4660,MainProcess):2025-10-24-19:26:48.488.255 [mindspore\common\api.py:125] The function "after_grad" at the file "c:\Users\AJ\anaconda3\envs\project\lib\site-packages\mindspore\ops\composite\base.py", line 597 has been compiled again. Try to reuse the function object decorated by @jit to reduce the compile time. For more details, get instructions about `jit` at https://www.mindspore.cn/search?inputValue=jit. 


[Full 9/12] Loss:0.0281 | Train:0.9975 | Val:0.9967 | 265.4s


RuntimeError: could not create a primitive

In [26]:
# --- 8. Save Checkpoint ---
os.makedirs("../models", exist_ok=True)
ms.save_checkpoint(model, "../models/resnet50_food.ckpt")
print("✅ Training complete. Checkpoint saved to ../models/resnet50_food.ckpt")


✅ Training complete. Checkpoint saved to ../models/resnet50_food.ckpt


In [25]:
ms.save_checkpoint(model, "../models/resnet50_food_last.ckpt")
with open("../models/labels.json", "w") as f:
    json.dump(classes, f, indent=2)
print("Saved last checkpoint and labels.json")


Saved last checkpoint and labels.json
